# 알파 채널이 추가된 png 파일 만들기

### 프로그램 실행 방법

1. 편집하려는 이미지의 경로를 직접 입력 : 옳은 경로를 입력할 때까지 반복

2. 실행할 기능에 대한 버튼을 키보드로부터 입력 : [1,2,3,r,i,+,-,ESC,q,p,s] 중 입력
    * '1': 역투영
        * 드래그 한 영역에 대한 역투영
    * '2': 흑백 이미지 쓰레시 홀드
        * 't': 선택영역 반전 토글
        * 'm': 적응형/전역 쓰레시홀드 토글
        * 트랙바로 값을 조절하여 이진화
    * '3': hsv 마스킹
        * 클릭 좌표 픽셀로 마스킹
        * 트랙바로 조절하여 마스킹
    * 'r': roi 영역 선택
    * 'i': 이미지 초기화
    * '+': 이미지 확대
    * '-': 이미지 축소
    * 'ESC', 'q': 종료
    
    (아직 미구현)
    * 'p': 미리보기
    * 's': 저장

In [1]:
# roi를 선택해 img_copy에 저장하는 역할을 수행하는 함수
def roi_copy(window_name):
    global img_copy
    
    print("roi 영역 선택")
    x,y,w,h = cv2.selectROI(window_name, img_copy, False) # selectROI OpenCV 함수
    if w and h:
        roi = img_copy[y:y+h, x:x+w] # roi 영역 referencing
        img_copy = roi
        key = cv2.waitKey(0)
    cv2.destroyWindow(window_name)


# 역투영 결과를 마스킹해서 결과를 보여주는 함수
def masking(img_backproject, window_name, img_copy):
    # 표면을 부드럽게 해주는 코드
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    cv2.filter2D(img_backproject, -1, disc, img_backproject)
    
    # 쓰레시 홀딩
    _, mask_backproject = cv2.threshold(img_backproject, 1, 255, cv2.THRESH_BINARY)
    img_result = cv2.bitwise_and(img_copy, img_copy, mask=mask_backproject) # bitwise 연산으로 마스크에 해당하는 좌표값만 추출
    cv2.imshow(window_name, img_result)
    cv2.waitKey()
    cv2.destroyWindow(window_name)
    
    
def back_project(window_name, img_copy):
    print("역투영")
    
    img_hsv = cv2.cvtColor(img_copy, cv2.COLOR_BGR2HSV)
    img_result = img_copy.copy()
    #cv2.imshow(window_name, img_copy)
    #cv2.moveWindow(window_name, 700, 5)  # 창 위치 변경
    
    x,y,w,h = cv2.selectROI(window_name, img_copy, False) # selectROI OpenCV 함수
    if w > 0 and h > 0:
        roi = img_result[y:y+h, x:x+w]
        roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) # roi를 hsv 형식으로 변환
        roi_hist = cv2.calcHist([roi_hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] ) # hue값, saturation값에 대한 히스토그램 계산
        
        img_backproject = cv2.calcBackProject([img_hsv], [0, 1], roi_hist,  [0, 180, 0, 256], 1) # OpenCV 역투영 함수 호출하여 결과를 저장
        masking(img_backproject,'img_result', img_copy) # 저장한 결과를 매개변수로 주어 마스킹 함수 호출
    cv2.destroyWindow('Back Project')
        
        
                   
# 쓰레시홀드 트랙바 생성을 위한 콜백함수
def for_threshold_trackbar(x):
    pass # 아무일도 수행하지 않음 - 반복문에서 수행


# hsv 값에 대한 트랙바 생성을 위한 콜백함수
def for_hsv_trackbar(x):
    pass # 아무일도 수행하지 않음 - 반복문에서 수행


#  마스킹 창에서 마우스 클릭시 호출되는 함수
def mouse_callback_masking(event, x, y, flags, param): 
    # 왼쪽 클릭 했을 때
    if event == cv2.EVENT_LBUTTONDOWN:
        # 해당 좌표의 픽셀값(bgr)을 받아와서, shape[3]을 [1][1][3]형식으로 변환하여 저장
        pixel = np.uint8([[img_copy[y,x]]]) 
        hsv = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV) # BGR 형식을 HSV 형식으로 변환
        
        # 선택한 좌표의 hsv 값들로 트랙바의 값을 셋팅
        cv2.setTrackbarPos('Hue', 'img_result', hsv[0][0][0]) # 색상(Hue) 세팅
        cv2.setTrackbarPos('Saturation', 'img_result', hsv[0][0][1]) # 채도(Saturation) 셋팅
        cv2.setTrackbarPos('Value', 'img_result', hsv[0][0][2]) # 밝기(Value) 셋팅


import cv2
import numpy as np

# 해당 경로에 이미지가 존재할 때 까지 반복
while True:
    img_file = input("이미지 경로 :") #'img/t1.jpg' #
    img = cv2.imread(img_file)
    if img is None:
        print("경로에 이미지가 존재하지 않습니다.")
        continue
        
    ############### BGR2BGRA로 바꾸는걸 저장할 때, 미리보기 할 때로 변경하기 ###############
    ###############   대신 img_copy, img_mask(alpha용), img_result를 사용   ###############
    
    img_copy = img.copy()
    break

# 프로세스 수행 반복, 키보드 입력으로 다음 기능을 선택
while True:
    print("1: 역투영\n2: 쓰레시 홀드\n3: hsv 마스킹\nr: roi 영역 선택")
    print("+: 이미지 확대\n-: 이미지 축소\ns: 저장\nESC, q: 종료\n")

    cv2.namedWindow('Current IMG')         # Current IMG 이름으로 창 생성
    cv2.imshow('Current IMG', img_copy)
    #cv2.moveWindow('Current IMG', 700, 5)  # 창 위치 변경
    
    # 키보드 입력으로 작업 선택 [esc,+,-,1,2,3,i,q,r,s]
    while True:
        key = cv2.waitKey(0) & 0xFF
        print("입력", key, chr(key))
        # ascii code {esc,+,-,1,2,3,i,q,r,s}
        if key in {27,43,45,49,50,51,105,113,114,115}: 
            cv2.destroyWindow('Current IMG')
            break
    
    # q, esc - 닫기
    if key == ord('q') or key == ord(''): # to ascii code
        print("key = ", key)
        break

    # + - 그림 확대
    elif key == ord('+'): # 배율 지정으로 확대, CUBIC
        print("그림 확대")
        img_copy = cv2.resize(img_copy, None, None, 2, 2, interpolation = cv2.INTER_CUBIC)

    # - - 그림 축소
    elif key == ord('-'): # 크기 지정으로 축소, AREA
        print("그림 축소")
        img_copy = cv2.resize(img_copy, None, None, 0.5, 0.5, interpolation = cv2.INTER_AREA)
#        img_copy = cv2.resize(img_copy, (int(img_copy.shape[0]*0.5), int(img_copy.shape[1]*0.5)), interpolation = cv2.INTER_AREA)
        
    # i - Current IMG 초기 이미지로 초기화
    elif key == ord('i'):
        print("초기화")
        img_copy = img.copy()

    # s - 저장
    elif key == ord('s'):
        print("저장")
        cv2.destroyWindow('Current IMG')
        
        # 알파 채널이 존재하는 BGRA 형식으로 변환
        img_result = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2BGRA)
        img_result[:, :, [3]] = 0 # 알파채널을 전부 0으로 초기화
        # img_result = bitwise_and(img_result, img_result, mask=img_mask)
        
        ###################### 저장 알고리즘 구현 필요 ######################
        ###################### 지금은 그냥 그대로 저장 ######################
        
        cv2.imwrite(img_file[:-3]+"png", img_copy)

    # r - roi 영역 선택
    elif key == ord('r'):
        roi_copy('ROI Select')
    
    # 1번 - 역투영 방식 선택했을 때
    elif key == ord('1'):
        back_project('Back Project', img_copy)
        ###################### 역투영 방식 구현중 ######################
        
        
    # 2번 - 흑백 사진에 대한 쓰레시 홀드를 통한 이분화작업 수행
    elif key == ord('2'):
        print("이진 쓰레시홀드")
        cv2.namedWindow('Binary Threshold') # 창 이름 설정
        cv2.createTrackbar('value', 'Binary Threshold', 0, 255, for_threshold_trackbar) # 0~255 값 트랙바 생성
        cv2.setTrackbarPos('value', 'Binary Threshold', 127)   # 트랙바 초기값 127 설정
        
        img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY) # 이미지 흑백으로 변환
        img_temp = np.zeros_like(img_copy)                     # 이미지와 같은 크기로 임시 배열 만들기
        flag = cv2.THRESH_BINARY                               # 초기 flag 설정
        
        blockSize = 3       # 블럭 사이즈 초기값 (blockSize % 2 == 1 && blockSize > 1 이어야 함)
        C = 0               # 차감 상수 초기값
        threshold = True   # 쓰레시홀드 전역 / 적응형 선택 변수
        
        # 쓰레시홀드 연산 마칠 때 까지 무한 반복
        while True:
            if threshold: # 전역 쓰레시홀드
                value = cv2.getTrackbarPos('value', 'Binary Threshold')   # 임계값 받아오기
                _, img_binary = cv2.threshold(img_gray, value, 255, flag) # 쓰레시 홀드 
                cv2.imshow('Binary Threshold', img_binary)
                #cv2.moveWindow('Binary Threshold', 700, 5)  # 창 위치 변경
                
            else:         # 적응형 쓰레시홀드
                size = cv2.getTrackbarPos('blockSize', 'Binary Threshold2')  # 블럭 사이즈
                if size%2 and size>1:   # blockSize % 2 == 1 && blockSize > 1 조건 불충족 시 오류 발생
                    blockSize = size
                C = cv2.getTrackbarPos('C', 'Binary Threshold2')             # 차감 상수 

                img_binary = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, flag, blockSize, C)
                cv2.imshow('Binary Threshold2', img_binary)
                #cv2.moveWindow('Binary Threshold2', 700, 5)  # 창 위치 변경

            # non-block 상태로, 입력이 있는지 확인해서 있으면 아래 조건문 검사
            key = cv2.waitKey(1)&0xFF
            print(key)
            if key != 255: # 입력이 있을 경우에만 (불필요한 조건 검사 줄이기)
                # esc, q - 닫기
                if key == 27 or key == 113:
                    break
                
                # s - 저장
                elif key == ord('s'):
                    print("저장")
                    
                    ###################### 저장 알고리즘 구현 필요 ######################
                    ###################### 지금은 그냥 그대로 저장 ######################
                    
                    cv2.imwrite(img_file[:-3]+"png", img_binary)
                    break
                    
                # t - 쓰레시홀드 결과값 반전 토글형식으로 선택
                elif key == ord('t'):
                    if flag == cv2.THRESH_BINARY:
                        flag = cv2.THRESH_BINARY_INV # THRESH_BINARY_INV : 반전된 마스크 이미지
                    else:
                        flag = cv2.THRESH_BINARY # cv2.THRESH_BINARY: 임계값 이상은 value로, 미만은 0으로 지정
                        
                # m - 쓰레시홀드 전역 / 적응형 토글형식으로 선택
                elif key == ord('m'):
                    if threshold:
                        threshold = False
                        cv2.destroyWindow('Binary Threshold')
                        cv2.namedWindow('Binary Threshold2') # 창 이름 설정
                        cv2.createTrackbar('blockSize', 'Binary Threshold2', 0, 255, for_threshold_trackbar) # 트랙바 생성
                        cv2.createTrackbar('C', 'Binary Threshold2', -50, 50, for_threshold_trackbar) # 트랙바 생성
                        cv2.setTrackbarPos('blockSize', 'Binary Threshold2', 0) # 트랙바 초기값 설정
                        cv2.setTrackbarPos('C', 'Binary Threshold2', 0) # 트랙바 초기값 설정
                    else:
                        threshold = True
                        cv2.destroyWindow('Binary Threshold2')
                        cv2.namedWindow('Binary Threshold') # 창 이름 설정
                        cv2.createTrackbar('value', 'Binary Threshold', 0, 255, for_threshold_trackbar) # 트랙바 생성
                        cv2.setTrackbarPos('value', 'Binary Threshold', 127) # 트랙바 초기값 설정
        cv2.destroyWindow('Binary Threshold')
        cv2.destroyWindow('Binary Threshold2')
    
    
    # 3번 - hsv 마스킹 방식 눌렀을 때
    elif key == ord('3'):
        
        hsv = 0 # hue, saturation, value 값
        lower_hsv = 0 # 범위 최소
        upper_hsv = 0 # 범위 최대
        offset = 20 # default offset
        
        cv2.namedWindow('hsv Masking') # 값을 뽑아낼 영상의 창 이름
        cv2.namedWindow('img_result') # 각 값으로 뽑아낸 결과 출력창 이름
        cv2.setMouseCallback('hsv Masking', mouse_callback_masking) # 마우스 콜백 함수 등록

        cv2.createTrackbar('Hue', 'img_result', 0, 180, for_hsv_trackbar)
        cv2.createTrackbar('Saturation', 'img_result', 0, 255, for_hsv_trackbar)
        cv2.createTrackbar('Value', 'img_result', 0, 255, for_hsv_trackbar)
        cv2.createTrackbar('Offset', 'img_result', 0, 100, for_hsv_trackbar)
        cv2.setTrackbarPos('Hue', 'img_result', 0)
        cv2.setTrackbarPos('Saturation', 'img_result', 0)
        cv2.setTrackbarPos('Value', 'img_result', 0)
        cv2.setTrackbarPos('Offset', 'img_result', 20)


        while True:
            hue = cv2.getTrackbarPos('Hue', 'img_result')
            saturation = cv2.getTrackbarPos('Saturation', 'img_result')
            value = cv2.getTrackbarPos('Value', 'img_result')
            offset = cv2.getTrackbarPos('Offset', 'img_result')

            # hue값과 유사한 픽셀값의 범위 설정, hue값 +- offset
            lower_hsv = np.array([hue-offset,saturation-offset,value-offset])
            upper_hsv = np.array([hue+offset,saturation+offset,value+offset])

            img_hsv = cv2.cvtColor(img_copy, cv2.COLOR_BGR2HSV) # BGR에서 HSV형식으로 변환

            # 유사한 범위 안에 해당하는 값은 그대로, 아닌 부분은 0으로 바꾸어 반환
            img_mask = cv2.inRange(img_hsv, lower_hsv, upper_hsv)

            # img_copy 에 대하여 img_mask 각 요소 값이 0이 아닌 픽셀들만 골라내는 연산
            img_result = cv2.bitwise_and(img_copy, img_copy, mask=img_mask)

            cv2.imshow('hsv Masking', img_copy)
            #cv2.moveWindow('hsv Masking', 730, 0)  # 창 위치 변경
            cv2.imshow('img_mask', img_mask)
            cv2.imshow('img_result', img_result)
            #cv2.moveWindow('img_result', 0, 0)  # 창 위치 변경

            if cv2.waitKey(1) & 0xFF == 27:
                break

        cv2.destroyWindow('hsv Masking')
        cv2.destroyWindow('img_mask')
        cv2.destroyWindow('img_result')

print("종료합니다.")

이미지 경로 :img/s1.jpg
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
r: roi 영역 선택
+: 이미지 확대
-: 이미지 축소
s: 저장
ESC, q: 종료

입력 49 1
역투영
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
r: roi 영역 선택
+: 이미지 확대
-: 이미지 축소
s: 저장
ESC, q: 종료

입력 50 2
이진 쓰레시홀드
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
25

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
